In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd


# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
from cryptography.fernet import Fernet
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v, dtype = "str")) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False) # store values to file "phone_numbers.csv"

### Edit cell below to change phone number count

In [4]:
# set phone number count
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

In [5]:
# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

#shuffle phone number lists
random.shuffle(gojek_phone_numbers)
random.shuffle(grab_phone_numbers)

# write phone numbers to csv file
phone_number_storage_manager(gojek_phone_numbers, grab_phone_numbers)

# Define classes for the necessary for the algorithm

In [6]:
# class to generate numbers required for psi
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        # method to generate p, q and factors of p-1
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q] # since q is prime, 2q only has these 3 factors excluding itself
                                    # 2q = p-1
                                    # factors of p-1 required to compute order of generators (lagrange theorem)
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        # method to generate safe prime for p
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            # the method used from the cryptography already generates a safe prime, this portion is merely to double confirm
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    
    def generate_random_number(self, size):
        # method to generate client's secret
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation implemented in pow(x,y,z)
                                                             # python integers have arbitrary precisions, 
                                                             # no overflow would occur if operations done in pure python
            if (result == 1):
                return False # if candidate exponent any of the factors (1,2,q) congruent to 1modp, candidate is not a primitive
                             # generator
        return True # order of candidate == p-1, therefore candidate is a primitive generator

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False) # store data to file with filename

                


    
        

# Create psi client class

In [7]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1, fernet_key):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p # prime modulus
        self.factors_pminus1 = factors_pminus1 # to calculate order of hashed phone numbers, 
                                               # since algorithm requires them to be primitive generators
        
        # for symmetric encryption
        self.fernet_key = fernet_key
        self.f = Fernet(fernet_key)
        
        # values to track for psi algorithm
        self.my_hashed_set = None # h(x)
        self.my_self_encrypted_set = None # (h(x)^(my_secret))modp
        self.my_encrypted_set = None # (h(x)^(my_secret)(other_party_secret))modp
        self.other_party_encrypted_set = None # (h(y)^(my_secret)(other_party_secret))modp
        self.common_values = None # common phone numbers
        
        
        # create datafile for communication with another party
        
        # content to store in file
        # only need to share self_encrypted_values and other_party_encrypted_values
        # common_values to ensure both calculate the same intersection
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        # filenames
        self.name = name
        self.filename = name + "_data_v1.1.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.1.csv"
        
        # create file
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element): 
        # method to hash phone numbers to primitive root modulo p i.e. primitive generator

        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() # sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            # repeatedly hash until primitive root modulo p is obtained
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
                
        return hash_int
    
    def modular_exponentation(self, element):
        # compute (element^(private_key))modp

        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        # hash all phone numbers in my set to primitive root modulo p, one by one

        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        # encrypt all elements in a given set using private_key, one by one
        
        # two scenarios to consider
        # one: encrypt set sent by the other party
        if (is_other_party):
            decrypted_other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in decrypted_other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        # two: encrypt my own set
        else:
            set_to_encrypt = self.my_hashed_set
        
        # encrypt values in given set, one by one
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        # assign the encrypted set to the correct variable
        # update csv file used for communication
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
                    
        
    def get_intersection(self):
        # get intersection
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") # read my encrypted set from the other party's file
        my_encrypted_set_int = []
        
        # convert read values to integer
        for element in my_encrypted_set:
            my_encrypted_set_int.append(int(element))
            
        # assign to correct variable
        self.my_encrypted_set = my_encrypted_set_int
        
        # get intersection
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        # find the index of the elements in the intersection in my_encrypted_set
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        # find the values in my own set corresponding to the index of the elements in the intersection
        for index in index_of_common_values:
            self.common_values.append(self.my_set[index])
            
        # update csv file for communication
        self.send_data(self.common_values, "common_values")
    
    def encrypt_data(self, plaintext):
        # encrypt data with Fernet
        
        endian = "big"
        element = plaintext.to_bytes(128, endian) # 1024 bits == 128 bytes
        cipher_text = self.f.encrypt(element)
        return cipher_text
    
    def decrypt_data(self, ciphertext):
        # decrypt data encrypted by Fernet
        
        ciphertext_bytes = ciphertext.encode('utf-8')[2:-1] # convert from string back to bytes
        endian = "big"
        element_in_bytes = self.f.decrypt(ciphertext_bytes)
        plaintext = int.from_bytes(element_in_bytes, endian)
        return plaintext

    def send_data(self, data_to_send, column_name):
        # send data means writing to file. encrypt data with Fernet
        
        # encrypt data
        encrypted_data_to_send = []
        for element in data_to_send:
            encrypted_element = self.encrypt_data(element)
            encrypted_data_to_send.append(encrypted_element)
            
        # send data
        self.my_dict[column_name] = encrypted_data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file (my file). decrypt data encrypted by Fernet
        
        # receive data
        encrypted_data = self.get_other_party_data()[column_name].to_list()
        
        # decrypt data
        decrypted_data = []
        for element in encrypted_data:
            if type(element) is float: # remove NaN
                continue
            decrypted_element = self.decrypt_data(element)
            decrypted_data.append(decrypted_element)
        print(f"number of elements in {self.other_party_name};{column_name}: {len(decrypted_data)}.")
        return decrypted_data
    
    def get_my_data(self):
        # read my file as dataframe (other party's file)
        
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        # read other party's file as dataframe
        
        return pd.read_csv(self.other_party_filename)
    


# Initialize context

### Edit cell below to change key size

In [8]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

In [9]:
# create key for symmetric key cryptography
fernet_key = Fernet.generate_key()

# create public parameters required for psi
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_phone_numbers, p, factors_pminus1, fernet_key)
gojek = Client("gojek", "grab", key_size, gojek_phone_numbers, p, factors_pminus1, fernet_key)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_9980/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers

In [10]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Clients' status after step 1 (value of variables in client)
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,1061642916839729595311745988861392474934466737...,3565755568280639079050051854599706700190867755...
1,8092639001204638790178908996283896242788771916...,2369003795800176044803624843724327263177283800...
2,2953623139145187618071647113313426330528576167...,3003360056476023280396172869223514448918734055...
3,1045158237184514971592009471668554581733135886...,1017141269956754882131908776753968058318200171...
4,4320682999770379726688099447684191745405506617...,5750822890154339790428118545565634695520281701...
...,...,...
86,NaN,3922283845005472934707328127251028806214862373...
87,NaN,3683371775181761031500661295222456997304332645...
88,NaN,8119100922771943066177300284568368128847410924...
89,NaN,5011913842513548422134267800545138946266586949...


## Step 2: encrypt hashed set with own private key

In [12]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_9980/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [13]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,1061642916839729595311745988861392474934466737...,3565755568280639079050051854599706700190867755...,8216034791276970847306380633948325273202324624...,1473391042748278319464361526553153363734857802...
1,8092639001204638790178908996283896242788771916...,2369003795800176044803624843724327263177283800...,6180514018684228413883769591028051438340544460...,8814153511784753629571064371334482478584949567...
2,2953623139145187618071647113313426330528576167...,3003360056476023280396172869223514448918734055...,8585246969366826828661580664709005780680762714...,2837131679255391884956628808072673484055701806...
3,1045158237184514971592009471668554581733135886...,1017141269956754882131908776753968058318200171...,1190595371927793407865212779961177538108047401...,8556768051348501773096812204361847698861452639...
4,4320682999770379726688099447684191745405506617...,5750822890154339790428118545565634695520281701...,8464022821308580592694159016860397460110634121...,2015239893501049512669077204757190490270743772...
...,...,...,...,...
86,NaN,3922283845005472934707328127251028806214862373...,NaN,1106775524038592919721748769021247475015950734...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,6026902457150574286495146306445921650751058504...
88,NaN,8119100922771943066177300284568368128847410924...,NaN,1086469059136491752544121845908759485591174109...
89,NaN,5011913842513548422134267800545138946266586949...,NaN,2834979477414145924870803592949719290115476257...


### Files' status after step 2 (value of variables in file - clients' variables encrypted with Fernet)

In [14]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfXYvE5W96RPDLMBJt3EaYFg-WIhEq6L78M...,NaN,NaN
1,b'gAAAAABhoxfXwt0XagpZbIigVCWnNlbQ2GLN7iNwYtXI...,NaN,NaN
2,b'gAAAAABhoxfXsz9UBDcpTJ8Uo1SQa6syqGu74cyANY6R...,NaN,NaN
3,b'gAAAAABhoxfXGXfbGD5-l5Fv6xwFXQoLZh6bLjbQ3If4...,NaN,NaN
4,b'gAAAAABhoxfXsaivn2IaFg65qgmBB7mFmMq0hMr414GF...,NaN,NaN
...,...,...,...
56,b'gAAAAABhoxfX7ChF7RgYaQBa16l5Gp70lyHQVvR9VpDi...,NaN,NaN
57,b'gAAAAABhoxfXPkquocfYRtgpE7Rdq74QScHC7GhxZaRX...,NaN,NaN
58,b'gAAAAABhoxfX9fHUAhnbZ2uuoLv_08-iLfXv71gjEkPl...,NaN,NaN
59,b'gAAAAABhoxfXP7dM1K3XxHiNAmtqZ-10RhBpaaXAo1oB...,NaN,NaN


In [15]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfXcFvTmB4lU-NC7GrwB_kGM6goin9uXoTj...,NaN,NaN
1,b'gAAAAABhoxfX77_gYikR7eTVwUtSfnEZXH0zu8o-4qwL...,NaN,NaN
2,b'gAAAAABhoxfXV5KTNZ0z2MMLRiAR7eiGm5IJRUXuSK_b...,NaN,NaN
3,b'gAAAAABhoxfXm45JdHrya6eNpSDUpBL-HSahi2QkQQXV...,NaN,NaN
4,b'gAAAAABhoxfXVGzzDJ4wgyHhgyxzuhMzvwSk3WldUjLn...,NaN,NaN
...,...,...,...
86,b'gAAAAABhoxfXYokpEFfnlm8-i_ijHiwaYZCy8aCaFytm...,NaN,NaN
87,b'gAAAAABhoxfXc98z56bD0OLO2XKI8BnMDn4CkbCPlXQh...,NaN,NaN
88,b'gAAAAABhoxfXvQ0IXrNdE7pFcirAUFJW90F2YArCD3wo...,NaN,NaN
89,b'gAAAAABhoxfXYg91SfNY47_MUVF4i0hsjm35AEjVzydx...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key


In [16]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 61.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_9980/964370003.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 91.


### Clients' status after step 3

In [17]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,1061642916839729595311745988861392474934466737...,3565755568280639079050051854599706700190867755...,8216034791276970847306380633948325273202324624...,1473391042748278319464361526553153363734857802...,7290035415370895630237194911043645039662822170...,4998696143216596559183800090446947867269838580...
1,8092639001204638790178908996283896242788771916...,2369003795800176044803624843724327263177283800...,6180514018684228413883769591028051438340544460...,8814153511784753629571064371334482478584949567...,7775514315077140749858559869855784004823874768...,1640406245715145126642990815813166380958172368...
2,2953623139145187618071647113313426330528576167...,3003360056476023280396172869223514448918734055...,8585246969366826828661580664709005780680762714...,2837131679255391884956628808072673484055701806...,5703142421884093604955534391736176836948373066...,5731569530681913731845067569455301403587147991...
3,1045158237184514971592009471668554581733135886...,1017141269956754882131908776753968058318200171...,1190595371927793407865212779961177538108047401...,8556768051348501773096812204361847698861452639...,2961940083979764512845983496453574478496779497...,8111892746016657384176471675378519070643693877...
4,4320682999770379726688099447684191745405506617...,5750822890154339790428118545565634695520281701...,8464022821308580592694159016860397460110634121...,2015239893501049512669077204757190490270743772...,7858023304647787141409852483194007440438008985...,1085172814847245587130720880920862464299791004...
...,...,...,...,...,...,...
86,NaN,3922283845005472934707328127251028806214862373...,NaN,1106775524038592919721748769021247475015950734...,NaN,8868381511901255217182400978917556866346838797...
87,NaN,3683371775181761031500661295222456997304332645...,NaN,6026902457150574286495146306445921650751058504...,NaN,9734863876964173420914092843244838034255028644...
88,NaN,8119100922771943066177300284568368128847410924...,NaN,1086469059136491752544121845908759485591174109...,NaN,2001317508875151477566716021188282790622585843...
89,NaN,5011913842513548422134267800545138946266586949...,NaN,2834979477414145924870803592949719290115476257...,NaN,7201139847237101679814491189188379877420841264...


### Files' status after step 3

In [18]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek



gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfXYvE5W96RPDLMBJt3EaYFg-WIhEq6L78M...,b'gAAAAABhoxfYYWlmQW6IKtRHbosPg8MsX9JVM7UN3Nj3...,NaN
1,b'gAAAAABhoxfXwt0XagpZbIigVCWnNlbQ2GLN7iNwYtXI...,b'gAAAAABhoxfYdw8KyrIXMt8I-H25lIevIl53dwenTODn...,NaN
2,b'gAAAAABhoxfXsz9UBDcpTJ8Uo1SQa6syqGu74cyANY6R...,b'gAAAAABhoxfYkPCgUOufOuJqPw6Z2ZgsGKzyuUHk03vN...,NaN
3,b'gAAAAABhoxfXGXfbGD5-l5Fv6xwFXQoLZh6bLjbQ3If4...,b'gAAAAABhoxfYV6EUnKqrxIko7_PlQAN0iscAjhUgPWkJ...,NaN
4,b'gAAAAABhoxfXsaivn2IaFg65qgmBB7mFmMq0hMr414GF...,b'gAAAAABhoxfYprhimvQzQp7Iax72SLgZnPfNCO9eoKMT...,NaN
...,...,...,...
86,NaN,b'gAAAAABhoxfYgxwpcwnB45UveSF7ZUoZPMBwEcSAa4P5...,NaN
87,NaN,b'gAAAAABhoxfYUcwEKIn88UjNTo-AQiOpqGEk2qJBpV97...,NaN
88,NaN,b'gAAAAABhoxfYbMqj_XbmzajpFWnzmxjm_NBUFuElq24U...,NaN
89,NaN,b'gAAAAABhoxfYlqK-J1KeS_pcdgW7hDYJWssbQgq6dMNw...,NaN


In [19]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfXcFvTmB4lU-NC7GrwB_kGM6goin9uXoTj...,b'gAAAAABhoxfYjGhl2KkuDeNPgrKmLb4fa_J1IuU0cpbq...,NaN
1,b'gAAAAABhoxfX77_gYikR7eTVwUtSfnEZXH0zu8o-4qwL...,b'gAAAAABhoxfYeydA11RcenhYCk_K8eR8tK9wFFqH9p1a...,NaN
2,b'gAAAAABhoxfXV5KTNZ0z2MMLRiAR7eiGm5IJRUXuSK_b...,b'gAAAAABhoxfYCDP7iNUApQvkyIdx9rQqkdcgjm87t6eC...,NaN
3,b'gAAAAABhoxfXm45JdHrya6eNpSDUpBL-HSahi2QkQQXV...,b'gAAAAABhoxfYUqEAHT8JobtIbbJR0zs0C9RYG1A_NTkP...,NaN
4,b'gAAAAABhoxfXVGzzDJ4wgyHhgyxzuhMzvwSk3WldUjLn...,b'gAAAAABhoxfYAfE1I98d8e3XDl-xSlQ2oFuwaJO0YZ0t...,NaN
...,...,...,...
86,b'gAAAAABhoxfXYokpEFfnlm8-i_ijHiwaYZCy8aCaFytm...,NaN,NaN
87,b'gAAAAABhoxfXc98z56bD0OLO2XKI8BnMDn4CkbCPlXQh...,NaN,NaN
88,b'gAAAAABhoxfXvQ0IXrNdE7pFcirAUFJW90F2YArCD3wo...,NaN,NaN
89,b'gAAAAABhoxfXYg91SfNY47_MUVF4i0hsjm35AEjVzydx...,NaN,NaN


## Step 4: find intersection


In [20]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 91.
number of elements in grab;other_party_encrypted_set: 61.


### Clients' status after step 4:

In [21]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,1061642916839729595311745988861392474934466737...,3565755568280639079050051854599706700190867755...,8216034791276970847306380633948325273202324624...,1473391042748278319464361526553153363734857802...,7290035415370895630237194911043645039662822170...,4998696143216596559183800090446947867269838580...,95521626.0,95521626.0
1,8092639001204638790178908996283896242788771916...,2369003795800176044803624843724327263177283800...,6180514018684228413883769591028051438340544460...,8814153511784753629571064371334482478584949567...,7775514315077140749858559869855784004823874768...,1640406245715145126642990815813166380958172368...,81093373.0,81093373.0
2,2953623139145187618071647113313426330528576167...,3003360056476023280396172869223514448918734055...,8585246969366826828661580664709005780680762714...,2837131679255391884956628808072673484055701806...,5703142421884093604955534391736176836948373066...,5731569530681913731845067569455301403587147991...,96485172.0,96485172.0
3,1045158237184514971592009471668554581733135886...,1017141269956754882131908776753968058318200171...,1190595371927793407865212779961177538108047401...,8556768051348501773096812204361847698861452639...,2961940083979764512845983496453574478496779497...,8111892746016657384176471675378519070643693877...,96192082.0,96192082.0
4,4320682999770379726688099447684191745405506617...,5750822890154339790428118545565634695520281701...,8464022821308580592694159016860397460110634121...,2015239893501049512669077204757190490270743772...,7858023304647787141409852483194007440438008985...,1085172814847245587130720880920862464299791004...,89312048.0,89312048.0
...,...,...,...,...,...,...,...,...
86,NaN,3922283845005472934707328127251028806214862373...,NaN,1106775524038592919721748769021247475015950734...,NaN,8868381511901255217182400978917556866346838797...,NaN,NaN
87,NaN,3683371775181761031500661295222456997304332645...,NaN,6026902457150574286495146306445921650751058504...,NaN,9734863876964173420914092843244838034255028644...,NaN,NaN
88,NaN,8119100922771943066177300284568368128847410924...,NaN,1086469059136491752544121845908759485591174109...,NaN,2001317508875151477566716021188282790622585843...,NaN,NaN
89,NaN,5011913842513548422134267800545138946266586949...,NaN,2834979477414145924870803592949719290115476257...,NaN,7201139847237101679814491189188379877420841264...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [22]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfXYvE5W96RPDLMBJt3EaYFg-WIhEq6L78M...,b'gAAAAABhoxfYYWlmQW6IKtRHbosPg8MsX9JVM7UN3Nj3...,b'gAAAAABhoxfYAq9DLyRzCtRnA8b-fhCd6KRsUe8pAFsh...
1,b'gAAAAABhoxfXwt0XagpZbIigVCWnNlbQ2GLN7iNwYtXI...,b'gAAAAABhoxfYdw8KyrIXMt8I-H25lIevIl53dwenTODn...,b'gAAAAABhoxfYExRy3I0_QeUYGZAaxqDBJkR9HK7gwH8e...
2,b'gAAAAABhoxfXsz9UBDcpTJ8Uo1SQa6syqGu74cyANY6R...,b'gAAAAABhoxfYkPCgUOufOuJqPw6Z2ZgsGKzyuUHk03vN...,b'gAAAAABhoxfYzPOWqqPBJx7r8wXXMCoruw25opk0zRCY...
3,b'gAAAAABhoxfXGXfbGD5-l5Fv6xwFXQoLZh6bLjbQ3If4...,b'gAAAAABhoxfYV6EUnKqrxIko7_PlQAN0iscAjhUgPWkJ...,b'gAAAAABhoxfYKFl_RlVVWTkBc-o1aX_5AOn3rzs573DD...
4,b'gAAAAABhoxfXsaivn2IaFg65qgmBB7mFmMq0hMr414GF...,b'gAAAAABhoxfYprhimvQzQp7Iax72SLgZnPfNCO9eoKMT...,b'gAAAAABhoxfYtZAdOW9SN2WsbZguFhbLtUuy_eJ7XRfN...
...,...,...,...
86,NaN,b'gAAAAABhoxfYgxwpcwnB45UveSF7ZUoZPMBwEcSAa4P5...,NaN
87,NaN,b'gAAAAABhoxfYUcwEKIn88UjNTo-AQiOpqGEk2qJBpV97...,NaN
88,NaN,b'gAAAAABhoxfYbMqj_XbmzajpFWnzmxjm_NBUFuElq24U...,NaN
89,NaN,b'gAAAAABhoxfYlqK-J1KeS_pcdgW7hDYJWssbQgq6dMNw...,NaN


In [23]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfXcFvTmB4lU-NC7GrwB_kGM6goin9uXoTj...,b'gAAAAABhoxfYjGhl2KkuDeNPgrKmLb4fa_J1IuU0cpbq...,b'gAAAAABhoxfYjj5x0tgd7-Nq12_VM9DnDX0pzwslevfc...
1,b'gAAAAABhoxfX77_gYikR7eTVwUtSfnEZXH0zu8o-4qwL...,b'gAAAAABhoxfYeydA11RcenhYCk_K8eR8tK9wFFqH9p1a...,b'gAAAAABhoxfYuzz4Gw3BXISmHKLl2NDiaE-_fa0oJ-cb...
2,b'gAAAAABhoxfXV5KTNZ0z2MMLRiAR7eiGm5IJRUXuSK_b...,b'gAAAAABhoxfYCDP7iNUApQvkyIdx9rQqkdcgjm87t6eC...,b'gAAAAABhoxfYU815umLAfmeoyTDyOqCFJQMGxvcUYRYS...
3,b'gAAAAABhoxfXm45JdHrya6eNpSDUpBL-HSahi2QkQQXV...,b'gAAAAABhoxfYUqEAHT8JobtIbbJR0zs0C9RYG1A_NTkP...,b'gAAAAABhoxfYn3-FV6F8aONkE-rBUgba67q6rXX6wOm9...
4,b'gAAAAABhoxfXVGzzDJ4wgyHhgyxzuhMzvwSk3WldUjLn...,b'gAAAAABhoxfYAfE1I98d8e3XDl-xSlQ2oFuwaJO0YZ0t...,b'gAAAAABhoxfY9VXuQQ30KseZJ6VqKAl_gm1uRZ7hKeRx...
...,...,...,...
86,b'gAAAAABhoxfXYokpEFfnlm8-i_ijHiwaYZCy8aCaFytm...,NaN,NaN
87,b'gAAAAABhoxfXc98z56bD0OLO2XKI8BnMDn4CkbCPlXQh...,NaN,NaN
88,b'gAAAAABhoxfXvQ0IXrNdE7pFcirAUFJW90F2YArCD3wo...,NaN,NaN
89,b'gAAAAABhoxfXYg91SfNY47_MUVF4i0hsjm35AEjVzydx...,NaN,NaN


# Check results

In [24]:
# get intersection found by the two parties
gojek_found_intersection = gojek.common_values
grab_found_intersection = grab.common_values


# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,actual,gojek,grab
0,80497694,80497694,80497694
1,81093373,81093373,81093373
2,86915509,86915509,86915509
3,89312048,89312048,89312048
4,94391128,94391128,94391128
5,95521626,95521626,95521626
6,96192082,96192082,96192082
7,96485172,96485172,96485172
8,99173089,99173089,99173089
9,99397525,99397525,99397525
